# Build RAG App Using Chroma and Gemini Pro

## What is Chroma?
Chroma is a vector database specifically designed to efficiently store and retrieve large volumes of data represented as numerical vectors or embeddings.

These embeddings, often generated by complex models like Large Language Models (LLMs), capture the semantic and syntactic meaning of text, images, or other data types. By storing these embeddings in Chroma, you can rapidly search and retrieve relevant information based on similarity rather than traditional keyword matching.    


## Why is Chroma Key for GenAI Applications?

GenAI applications, such as chatbots, question answering systems, and recommendation engines, rely heavily on the ability to process and understand vast amounts of information quickly and accurately. Chroma plays a critical role in this process by:

- Efficient Information Retrieval: Chroma’s vector-based search allows for rapid retrieval of relevant information based on semantic similarity, which is crucial for providing accurate and contextually relevant responses.   
- Knowledge Base Management: GenAI models often require access to a large knowledge base. Chroma can efficiently store and manage this knowledge, making it readily available for the model to access.   
- Real-time Performance: Chroma is optimized for speed, enabling real-time interactions with users. This is essential for applications that require immediate responses, such as chatbots or virtual assistants.
- Scalability: As the amount of data grows, Chroma can scale to handle increasing workloads, ensuring the continued performance of the GenAI application.

In essence, Chroma serves as the foundational component that empowers GenAI applications to deliver exceptional performance and user experiences. By effectively managing and accessing information, Chroma enables these applications to provide intelligent and informative responses.

## Installation


In [ ]:
!pip install chromadb

In [1]:
!pip list | grep chromadb

chromadb                                 0.5.0


## Chroma API demo


In [1]:
import chromadb

In [26]:
client = chromadb.PersistentClient(path="/mnt/c/dclab/dev/ai-ml/.chroma")
client.heartbeat()
#client.reset()

1722707385248475795

verify local storage

In [27]:
!ls -ltr /mnt/c/dclab/dev/ai-ml/.chroma

total 192
drwxrwxrwx 1 oldhorse oldhorse   4096 Aug  3 13:26 313c4718-1459-4846-aae5-ffb9b1bcfc92
-rwxrwxrwx 1 oldhorse oldhorse 147456 Aug  3 13:26 chroma.sqlite3


## Create collection

In [28]:
collection = client.get_or_create_collection(name="test")

In [29]:
collection = client.get_collection(name="test")
print(collection)

name='test' id=UUID('81de8271-d3f0-40f0-a9bb-fb709083c40e') metadata=None tenant='default_tenant' database='default_database'


In [30]:
collection.peek() 

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [31]:
collection.count()

0

In [40]:
collections = client.list_collections()
names = [collection.name for collection in collections]
names

['new_test', 'test']

## Add to collection

In [34]:
collection.add(
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)

In [35]:
collection.count()

3

In [38]:
collection.peek() 

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': [[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
 'metadatas': [{'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'},
  {'chapter': '29', 'verse': '11'}],
 'documents': [None, None, None],
 'uris': None,
 'data': None}

## Query collection

In [43]:
collection.query(
    query_embeddings=[[11.1, 12.1, 13.1],[1.1, 2.3, 3.2]],
    n_results=3,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
)

{'ids': [[], []],
 'distances': [[], []],
 'metadatas': [[], []],
 'embeddings': None,
 'documents': [[], []],
 'uris': None,
 'data': None}

In [46]:
collection.get(
	ids=["id1", "id2", "id3"],
	where={"chapter": "3"}
)


{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [{'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'}],
 'documents': [None, None],
 'uris': None,
 'data': None}

## Delete id from collection

In [50]:
collection.delete(
    ids=["id1", "id2", "id3"],
	where={"chapter": "29"}
)

In [51]:
collection.get(
	ids=["id1", "id2", "id3"]
)

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [{'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'}],
 'documents': [None, None],
 'uris': None,
 'data': None}

## Building a RAG App with Gemini Pro API

**Installation**

In [ ]:
!pip install -U -q google-generativeai   
!pip install -q chromadb

**Import packages**

In [52]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

import google.generativeai as genai

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

**Setup API key**
Put Gemini API key in .env under current folder,
```
GOOGLE_API_KEY=xxx
```

In [53]:
import os

from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

**list supported embedding models**

In [54]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


**Sample data**

In [56]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]
documents

['Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will 

**Embeddings with model embedding-001**

In [57]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

**Create chroma db**

In [58]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

**Setup vector db**

In [59]:
db = create_chroma_db(documents, "googlecarsdatabase")

**Verify db**

In [60]:
pd.DataFrame(db.peek(3))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[-0.020994942635297775, -0.03876612335443497, ...",None,Operating the Climate Control System Your Goo...,None,None
1,1,"[0.017410801723599434, -0.04757162556052208, -...",None,Your Googlecar has a large touchscreen display...,None,None
2,2,"[-0.03194405511021614, -0.023281503468751907, ...",None,Shifting Gears Your Googlecar has an automatic...,None,None


**Getting the relevant document**

In [61]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

**Perform embedding search**

In [63]:
passage = get_relevant_passage("touch screen features", db)
Markdown(passage)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

**Make a prompt**

In [64]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

**Pass a query to the prompt**

In [65]:
query = "How do you use the touchscreen in the Google car?"
prompt = make_prompt(query, passage)
Markdown(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you use the touchscreen in the Google car?'
  PASSAGE: 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the Navigation icon to get directions to your destination or touch the Music icon to play your favorite songs.'

    ANSWER:
  

**Generate a response**

In [66]:
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)
Markdown(answer.text)

Sure thing!  That's an excellent question.  Using the Google Car's touchscreen is easy, and I'll be happy to explain how it works. The Google Car has a large touchscreen display located in the center console of the vehicle. To interact with the touchscreen, simply touch the desired icon to access a variety of features such as navigation, entertainment, and climate control.  For example, you can touch the Navigation icon to get directions to your destination or touch the Music icon to play your favorite songs.